In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
driver = webdriver.PhantomJS()
#若直接抓取選擇權頁面的資料不知道為什麼會被期交所擋住，所以先抓期交所裡面的其他資料，在這之後再抓選擇權資料，並且把這行覆蓋掉。
driver.get('http://info512.taifex.com.tw/Future/FusaQuote_Norl.aspx')
#若直接抓取選擇權頁面的資料不知道為什麼會被期交所擋住，所以先抓期交所裡面的其他資料，在這之後再抓選擇權資料，並且把這行覆蓋掉。
soup = BeautifulSoup(driver.page_source,'lxml')
A = pd.read_html(str(soup.select('#divDG')[0]),header=0)[0].loc[8:20]
#把NA改成0啦!!
A.fillna(0)
#把--改成0啦!!
A.replace('--',0)
driver.get('http://info512.taifex.com.tw/Future/OptQuote_Norl.aspx')
selectbox = webdriver.support.ui.Select(driver.find_element_by_name('ctl00$ContentPlaceHolder1$ddlFusa_SelMon'))
selectbox.all_selected_options
[sel.text for sel in selectbox.options]
selectbox.select_by_value([sel.text for sel in selectbox.options][1])
soup = BeautifulSoup(driver.page_source,'lxml')
pd.read_html(str(soup.select('#divDG')[0]),header=0)[0].loc[8:20]
#將Call資料與Put資料分別放到兩個dataframe裡面
soup = BeautifulSoup(driver.page_source,"lxml")
table = pd.read_html(str(soup.select('#divDG')[0]))[0]
Call = table.loc[:,:6]
Put = table.loc[:,6:]
Call.columns = Call.loc[0]
Call = Call[1:]
Put.columns = Put.loc[0]
Put = Put[1:]
def Vol_conversion(input_ele):
    if input_ele == '--':
        return 0
    if input_ele!= '--':
        return int(''.join(input_ele.split(',')))
def mon_float(input_ele):
    if input_ele == '--':
        return None
    if input_ele!= '--':
        return float(''.join(input_ele.split(',')))
Call['總量'] = Call['總量'].map(Vol_conversion)
Put['總量'] = Put['總量'].map(Vol_conversion)
Call['成交'] = Call['成交'].map(mon_float)
Put['成交'] = Put['成交'].map(mon_float)
Call['履約價']=Call['履約價'].map(lambda x:int(x))
Put['履約價']=Put['履約價'].map(lambda x:int(x))
comp = np.array(Call['履約價'].tolist())+np.array(Call['成交'].tolist())-np.array(Put['成交'].tolist())
Call['組合價'] = pd.Series(comp,index=[Call.index])
Put['組合價'] = pd.Series(comp,index=[Put.index])
pd.Series(comp,index=[Call.index])
def EV(Spot_price,Strike_price):
    if Spot_price-Strike_price>0:
        return Spot_price-Strike_price
    else:
        return 0
Call['內含價值'] = Call.apply(lambda row: EV(row['組合價'],row['履約價']),axis=1)
Put['內含價值'] = Put.apply(lambda row: EV(row['組合價'],row['履約價']),axis=1)
Call['時間價值'] = Call.apply(lambda row: row['成交']-row['內含價值'] if row['成交']!= 0 else 0,axis=1)
Put['時間價值'] = Put.apply(lambda row: row['成交']-row['內含價值'] if row['成交']!= 0 else 0,axis=1)
driver.get('http://info512.taifex.com.tw/Future/FusaQuote_Norl.aspx?d=080585')
#先抓出現貨價格
soup = BeautifulSoup(driver.page_source,'lxml')
pd.read_html(str(soup.select('#divDG')[0]),header=0)[0].loc[0:0]
Spot_information = pd.read_html(str(soup.select('#divDG')[0]),header=0)[0].loc[0:0]
#給定期交所上的保證金A值以及B值
#賣出買權/賣出CALL : 權利金市值+MAXIMUM(A值-價外值，B值)
#賣出賣權/賣出PUT : 權利金市值+MAXIMUM(A值-價外值，B值)
A = 22000
B = 11000
S = float(Spot_information['成交價'])
spread1 = S-Call['履約價']
spread2 = Put['履約價']-S
Call["價外值"]=np.nan
for i in range(1,len(Call)):
    if spread1[i] > 0:
        Call["價外值"][i] = spread1[i] 
    else:
        Call["價外值"][i] = 0
Call['保證金']=np.nan
for i in range (1,len(Call)):
    Call['保證金'][i] = Call['成交'][i]*50 + max(A-Call["價外值"][i],B)
Put["價外值"]=np.nan
for i in range(1,len(Put)):
    if spread2[i] > 0:
        Put["價外值"][i] = spread2[i] 
    else:
        Put["價外值"][i] = 0 
Put['保證金']=np.nan
for i in range (1,len(Put)):
    Put['保證金'][i] = Put['成交'][i]*50 + max(A-Put["價外值"][i],B)
at_the_money = Call['履約價'] + Call['成交'] - Put['成交']
difference = at_the_money - Call['履約價']
df = pd.concat([at_the_money,Call['履約價'],Call['成交'],Call['保證金'],Put['成交'],Put['保證金']],axis = 1)
df.columns= ['K+C-P', 'Exercise_Price', 'Premium_Call', 'Margin_Call', 'Premium_Put', 'Margin_Put']
df[13:26]